01. Initializations

In [116]:
import pandas as pd
import sys
import json
import os
import datetime


root_folder = '../CREME_dataset'
scenario = ["mirai", "disk_wipe", "ransomware", "resource_hijacking", "end_point_dos"]
scenario_folder = ["01_mirai", "02_disk_wipe", "03_ransomware", "04_resource_hijacking", "05_end_point_dos"]
filename = "dataset_generation.log"
label = {'mirai': [1, 2, 3, 5, 9, 11, 12, 13],
         'disk_wipe': [1, 2, 4, 6, 8, 14],
         'ransomware': [1, 2, 4, 6, 7, 12, 15],
         'resource_hijacking': [1, 2, 4, 6, 8, 12, 16],
         'end_point_dos': [1, 2, 4, 8, 10, 12, 17]
         }

ips = {"controller": ["192.168.56.111"],
       "data-logger": ["192.168.56.121"],
       "attacker": ["192.168.56.131"],
       "non-vulnerable": ["192.168.56.141", "192.168.56.142"],
       "vulnerable": ["192.168.56.151"],
       "malicious": ["192.168.56.161"],
       "benign": ["192.168.56.171"],
       "target": ["192.168.56.181"]
       }

Test

In [117]:
df = pd.read_csv(os.path.join(root_folder, "original_label_syslog.csv"))#26940
df = df[df.HostName=="192.168.56.171"]
df = df[df.PID_or_IP.notnull()]
df = df[~df.PID_or_IP.str.contains('^localhost\] [^\[\]]+\[[0-9]+$')]
df.info()
df.describe()
df.head(100)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 52602 to 190485
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Time             2 non-null      object 
 1   HostName         2 non-null      object 
 2   Component        0 non-null      object 
 3   PID_or_IP        2 non-null      object 
 4   Content          2 non-null      object 
 5   EventId          2 non-null      object 
 6   EventTemplate    2 non-null      object 
 7   ParameterList    2 non-null      object 
 8   Timestamp        2 non-null      float64
 9   Label            2 non-null      int64  
 10  Tactic           2 non-null      object 
 11  Technique        2 non-null      object 
 12  SubTechnique     2 non-null      object 
 13  Label_lifecycle  2 non-null      int64  
dtypes: float64(1), int64(2), object(11)
memory usage: 240.0+ bytes


,Time,HostName,Component,PID_or_IP,Content,EventId,EventTemplate,ParameterList,Timestamp,Label,Tactic,Technique,SubTechnique,Label_lifecycle
52602,2023-03-06T17:33:49+08:00,192.168.56.171,NaN,localhost] rsyslogd: rsyslogd[internal_messages,98 messages lost due to rate-limiting (500 all...,7410c1db,<*> messages lost due to rate-limiting (500 al...,['98'],1.678095e+09,1,Reconnaissance,Active Scanning,Scanning IP Blocks,2
190485,2023-03-06T19:06:49+08:00,192.168.56.171,NaN,localhost] rsyslogd: rsyslogd[internal_messages,438 messages lost due to rate-limiting (500 al...,7410c1db,<*> messages lost due to rate-limiting (500 al...,['438'],1.678101e+09,1,Reconnaissance,Active Scanning,Scanning IP Blocks,5


2. Repairing the label

In [118]:
df = pd.read_csv(os.path.join(root_folder, "original_label_syslog.csv"))
del_list = ['Tactic', 'Technique', 'SubTechnique']
result_abs_path = os.path.join(root_folder, "modify")
# print(ips["benign"][0])

tmp_df = df[df.HostName=="192.168.56.171"]
tmp_df = tmp_df[tmp_df.PID_or_IP.notnull()]
tmp_df = tmp_df[tmp_df.PID_or_IP.str.contains('^localhost\] [^\[\]]+\[[0-9]+$')]
tmp_df.Component = tmp_df.PID_or_IP.str.extract('^localhost\] ([^\[\]]+)\[[0-9]+$')
tmp_df.PID_or_IP = tmp_df.PID_or_IP.str.extract('^localhost\] [^\[\]]+\[([0-9]+)$')

df.drop(df[df['HostName']=="192.168.56.171"].index, inplace=True)
df = pd.concat([df, tmp_df])

df.drop(df[df.Component.str.contains('apache-error')].index, inplace=True)

df.Label[df.HostName=="192.168.56.171"] = 0
df.Label_lifecycle[df.HostName=="192.168.56.171"] = 0
df['HostName'] = df['HostName'].replace('192.168.56.171', 'benign-server')
# print(df)
df = df.drop(columns=del_list)

if not os.path.exists(result_abs_path):
    os.makedirs(result_abs_path)
df.to_csv(os.path.join(result_abs_path, "original_relabel_syslog.csv"), index=False)

/var/folders/hg/c084_sjj0_1dhg3qjvswf_mh0000gn/T/ipykernel_9457/717144884.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Label[df.HostName=="192.168.56.171"] = 0
/var/folders/hg/c084_sjj0_1dhg3qjvswf_mh0000gn/T/ipykernel_9457/717144884.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Label_lifecycle[df.HostName=="192.168.56.171"] = 0


In [119]:
col_list = ['HostName', 'EventTemplate', 'Timestamp', 'Label', 'Label_lifecycle', 'PID_or_IP', 'Component']
one_hot_col_list = ['EventTemplate']

df = pd.read_csv(os.path.join(result_abs_path, "original_relabel_syslog.csv"), usecols=col_list)
labels = df['Label'].unique()

df = pd.get_dummies(df, columns=one_hot_col_list)

hostnames = df['HostName'].unique()

df_count_vector = pd.DataFrame()

df_machines = []
unique_timestamp_machines = []
for hostname in hostnames:
    df_machines.append(df[df['HostName'] == hostname])
    
for df_machine in df_machines:
    unique_timestamps = list(set(df_machine['Timestamp']))
    unique_timestamp_machines.append(unique_timestamps)

del_col_list = ['HostName', 'Timestamp']
for i in range(len(df_machines)):
    continue
    df_machine = df_machines[i]
    tmp_unique_timestamps = unique_timestamp_machines[i]
    for tmp_timestamp in tmp_unique_timestamps:
        tmp_df = df_machine[df_machine['Timestamp'] == tmp_timestamp]
        tmp_df = tmp_df.drop(columns=del_col_list)
        classes_num = {}
        for j in labels:
            class_num = len(tmp_df[tmp_df['Label'] == j])
            if j != 0 and class_num > 0:
                classes_num[j] = class_num
            sum_one_hot = tmp_df.sum()
            
        max_num = 0
        max_label = 0
        for key in classes_num:
            if classes_num[key] > max_num:
                max_num = classes_num[key]
                max_label = key
                
        sum_one_hot = pd.DataFrame(sum_one_hot).transpose()
        sum_one_hot['Label'] = max_label
        sum_one_hot['Timestamp'] = tmp_timestamp
        df_count_vector = pd.concat([df_count_vector, sum_one_hot], ignore_index=True)


if not os.path.exists(result_abs_path):
    os.makedirs(result_abs_path)
df.to_csv(os.path.join(result_abs_path, "relabel_syslog.csv"), index=False)

In [127]:
df = pd.read_csv(os.path.join(result_abs_path, "relabel_syslog.csv"))
# df = df[df.Component.str.contains('apache-error')]
df = df[df.HostName.isnull()]
# df = df[df.PID_or_IP.str.contains('^[0-9]+$')]
df.head(n=100)
# df.describe()
# df.PID_or_IP.unique()

/var/folders/hg/c084_sjj0_1dhg3qjvswf_mh0000gn/T/ipykernel_9457/1154463523.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(result_abs_path, "relabel_syslog.csv"))


,HostName,Component,PID_or_IP,Timestamp,Label,Label_lifecycle,"EventTemplate_""CLUR / HTTP/1.1"" 501 496 ""-"" ""Mozilla/5.0 (compatible; Nmap Scripting Engine; https://nmap.org/book/nse.html)""","EventTemplate_""FHWL / HTTP/1.1"" 501 496 ""-"" ""Mozilla/5.0 (compatible; Nmap Scripting Engine; https://nmap.org/book/nse.html)""","EventTemplate_""GET /chat/read_log.php HTTP/1.1"" 404 <*> ""about:blank"" ""Node.js (linux; U; rv:v4.9.1) AppleWebKit/537.36 (KHTML, like Gecko)""","EventTemplate_""GET <*> <*> 200 <*> ""-"" <*>",...,EventTemplate_pam_unix(samba:session): session closed for user nobody,EventTemplate_pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=<*> user=root,EventTemplate_pam_unix(sshd:session): session closed for user root,EventTemplate_pam_unix(sshd:session): session opened for user root by (uid=0),EventTemplate_statistics: max cache size <*> at Mar 6 <*>,EventTemplate_statistics: max connection <*> <*> for <*> at Mar 6 <*>,EventTemplate_warning: TLS library problem: <*> <*> <*>,EventTemplate_warning: TLS library problem: error:14094410:SSL routines:SSL3_READ_BYTES:sslv3 alert handshake failure:s3_pkt.c:1278:SSL alert number 40:,EventTemplate_warning: non-SMTP command from unknown[<*>]: <*>,EventTemplate_warning: unknown[<*>]: SASL NTLM authentication failed: Invalid authentication mechanism
